In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load



In [ ]:
# -*- coding: utf-8 -*-

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os



# In[1]加载数据集
train=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
test=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv')

# In[3]分词。将context+anchor作为sen1,target作为sentence2,计算其相似度,作为特征
#word_tokenize是英文的分词函数,但无法删除标点符号
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
#构建的tokenizer除了分词，还能去除标点符号
def filter_stopword(txt):
    sent=tokenizer.tokenize(txt)
    return sent
anc=list(train['anchor'])
tar=list(train['target'])
sen_lis_anchor=[filter_stopword(i) for i in anc]
sen_lis_tar=[filter_stopword(i) for i in tar]

# In[]
from gensim.models import word2vec
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = word2vec.Word2Vec(sen_lis_anchor+sen_lis_tar,min_count=1,epochs=5)

# In[]构造特征,计算句子相似度(词的相似度最大值再取均值)
def cal_sen_sim(s1,s2):
    results=[]
    for i in s1:
        result=0
        for j in s2:
            result=max(model.wv.similarity(i,j),result)
        results.append(result)
    return max(results)
fea=[cal_sen_sim(sen_lis_anchor[i],sen_lis_tar[i]) for i in range(len(sen_lis_anchor))]

# In[]
import numpy as np
X=np.array(fea).reshape(-1,1)
train=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv')
Y=train.score.replace([0,0.25,0.5,0.75,1],[1,2,3,4,5])
Y=Y.values.reshape(-1,1)
rate=0.8


trainX,testX=X[:int(len(X)*rate)],X[int(len(X)*rate):]
trainY,testY=Y[:int(len(Y)*rate)],Y[int(len(Y)*rate):]

#训练集中没有标签1,打乱顺序
indices=np.random.permutation(trainX.shape[0])
trainX_r=trainX[indices]
trainY_r=trainY[indices]
# In[]
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
knn.fit(trainX_r,trainY_r)
pred=knn.predict(testX)

# In[]]
pred[pred==1]=0
pred[pred==2]=0.25
pred[pred==3]=0.5
pred[pred==4]=0.75
pred[pred==5]=1
testY[testY==1]=0
testY[testY==2]=0.25
testY[testY==3]=0.5
testY[testY==4]=0.75
testY[testY==5]=1
print(np.corrcoef(pred,testY.squeeze())[0,1])

# In[]
anc_test=list(test['anchor'])
tar_test=list(test['target'])
sen_lis_anchor_test=[filter_stopword(i) for i in anc_test]
sen_lis_tar_test=[filter_stopword(i) for i in tar_test]
# In[]
fea_test=[cal_sen_sim(sen_lis_anchor_test[i],sen_lis_tar_test[i]) for i in range(len(sen_lis_anchor_test))]
# In[]
result=knn.predict(np.array(fea_test).reshape(-1,1))
# In[]
result[result==1]=0
result[result==2]=0.25
result[result==3]=0.5
result[result==4]=0.75
result[result==5]=1
sub=pd.read_csv('/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv')
sub['score']=result
sub.to_csv('/kaggle/working/submission.csv', index=False)